In [177]:
!pip install rdflib

In [0]:
import csv
import pprint
import random
import rdflib

In [0]:
INPUT_FILENAME = 'games.csv'
RDF_FILENAME = 'games.rdf'

# properties
_DBP_URI = 'dbp_uri'
_NAME = 'name'
_DEVELOPER = 'developer'
_PUBLISHER = 'publisher'
_RELEASE_DATE = 'release_date'
_HOMEPAGE = 'homepage'
_MODE = 'mode'
_IGN = 'ign'
_MC = 'metacritic'

#1. Make DB

In [180]:
INPUT_FILENAME = 'games.csv'

data = []

with open(INPUT_FILENAME, 'r', encoding='utf-8') as file:
  reader = csv.reader(file, delimiter=',', quotechar='"')
  headers = next(reader)
  for row in reader:
    data.append(dict(zip(headers, row)))
    
print(len(data))
pprint.pprint(data[:5])

1093
[{'dbp_uri': 'http://dbpedia.org/resource/Kingdom_Hearts_HD_1.5_Remix',
  'developer': 'http://dbpedia.org/resource/Square_Enix',
  'homepage': 'http://www.square-enix.co.jp/kingdom/khhd/',
  'ign': '85.0',
  'metacritic': '77',
  'mode': 'http://dbpedia.org/resource/Single-player_video_game',
  'name': 'Kingdom Hearts HD 1.5 Remix',
  'publisher': 'http://dbpedia.org/resource/Square_Enix',
  'release_date': '2013-03-14'},
 {'dbp_uri': 'http://dbpedia.org/resource/Project_Zero_2:_Wii_Edition',
  'developer': 'http://dbpedia.org/resource/Koei_Tecmo',
  'homepage': 'http://www.nintendo.co.jp/wii/sl2j/index.html',
  'ign': '85.0',
  'metacritic': '77',
  'mode': 'http://dbpedia.org/resource/Single-player_video_game',
  'name': 'Project Zero 2: Wii Edition',
  'publisher': 'http://dbpedia.org/resource/Nintendo',
  'release_date': '2012-06-28'},
 {'dbp_uri': 'http://dbpedia.org/resource/Planet_Minigolf',
  'developer': 'http://dbpedia.org/resource/Zen_Studios',
  'homepage': 'http://pl

In [181]:
lines = [
  '@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>',
  '@prefix dbo: <http://dbpedia.org/ontology/>',
  '@prefix dbr: <http://dbpedia.org/resource/>',
  '@prefix dbp: <http://dbpedia.org/property/>',
  '@prefix owl: <http://www.w3.org/2002/07/owl#>',
  '@prefix xsd: <http://www.w3.org/2001/XMLSchema#>',
]
lines.append('@prefix best: <http://thebestontology.com/>')

for game in data:
    game_uri = f'<http://thebestontology.com/{game[_NAME].replace(" ", "_")}>'
    
    lines.append(f'{game_uri} owl:sameAs <{game[_DBP_URI]}>')
    lines.append(f'{game_uri} rdfs:label "{game[_NAME]}"@en')

    lines.append(f'{game_uri} dbo:developer <{game[_DEVELOPER]}>')
    lines.append(f'{game_uri} dbo:publisher <{game[_PUBLISHER]}>')
    lines.append(f'{game_uri} dbo:releaseDate "{game[_RELEASE_DATE]}"^^xsd:date')
    lines.append(f'{game_uri} dbp:modes <{game[_MODE]}>')
    lines.append(f'{game_uri} dbp:ign {round(float(game[_IGN]))}')
    lines.append(f'{game_uri} dbp:mc {game[_MC]}')
    
    best_score = random.randint(1, 100)
    lines.append(f'{game_uri} best:score {best_score}')

for line in lines[:25]:
  print(line)

@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
@prefix dbo: <http://dbpedia.org/ontology/>
@prefix dbr: <http://dbpedia.org/resource/>
@prefix dbp: <http://dbpedia.org/property/>
@prefix owl: <http://www.w3.org/2002/07/owl#>
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>
@prefix best: <http://thebestontology.com/>
<http://thebestontology.com/Kingdom_Hearts_HD_1.5_Remix> owl:sameAs <http://dbpedia.org/resource/Kingdom_Hearts_HD_1.5_Remix>
<http://thebestontology.com/Kingdom_Hearts_HD_1.5_Remix> rdfs:label "Kingdom Hearts HD 1.5 Remix"@en
<http://thebestontology.com/Kingdom_Hearts_HD_1.5_Remix> dbo:developer <http://dbpedia.org/resource/Square_Enix>
<http://thebestontology.com/Kingdom_Hearts_HD_1.5_Remix> dbo:publisher <http://dbpedia.org/resource/Square_Enix>
<http://thebestontology.com/Kingdom_Hearts_HD_1.5_Remix> dbo:releaseDate "2013-03-14"^^xsd:date
<http://thebestontology.com/Kingdom_Hearts_HD_1.5_Remix> dbp:modes <http://dbpedia.org/resource/Single-player_video_game>
<htt

In [182]:
terminated_lines = [line+' .\n' for line in lines ]

rdf = ''.join(terminated_lines) + '\n'
with open(RDF_FILENAME, 'w') as file:
  file.write(rdf)
  
!ls -l

print(rdf[:500])

total 2104
-rw-r--r-- 1 root root 305550 May 31 09:36 games.csv
-rw-r--r-- 1 root root 919424 May 31 15:10 games.rdf
drwxr-xr-x 1 root root   4096 May 24 16:08 sample_data
-rw-r--r-- 1 root root 919424 May 31 12:23 test
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix dbp: <http://dbpedia.org/property/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix best: <http://thebestontology.com/> .
<http://thebestontology.com/Kingdom_Hearts_HD_1.5_Remix> owl:sameAs <http://dbpedia.org/resource/Kingdom_Hearts_HD_1.5_Remix> .
<http://thebestontology.com/Kingd


#2. Query

In [183]:
graph = rdflib.Graph()
graph.parse(RDF_FILENAME, format='n3')

<Graph identifier=Ncb3fec0fbd4a4c568b365229eceae0ed (<class 'rdflib.graph.Graph'>)>

In [184]:
query = """SELECT ?game ?name
WHERE {
?game rdfs:label ?name .
}
LIMIT 10
"""

query_result = graph.query(query)

last_row = None
for row in query_result:
  last_row = row
  print(f'{row[0]} | {row[1]}')

print()
print(last_row)

http://thebestontology.com/PlayStation_All-Stars_Battle_Royale | PlayStation All-Stars Battle Royale
http://thebestontology.com/Burn_Zombie_Burn | Burn Zombie Burn
http://thebestontology.com/Tiger_Woods_PGA_Tour_10 | Tiger Woods PGA Tour 10
http://thebestontology.com/Sly_3:_Honor_Among_Thieves | Sly 3: Honor Among Thieves
http://thebestontology.com/Onimusha_2:_Samurai's_Destiny | Onimusha 2: Samurai's Destiny
http://thebestontology.com/WinBack_2:_Project_Poseidon | WinBack 2: Project Poseidon
http://thebestontology.com/The_Temple_of_Elemental_Evil_(video_game) | The Temple of Elemental Evil (video game)
http://thebestontology.com/The_Amazing_Spider-Man_2_(2014_video_game) | The Amazing Spider-Man 2 (2014 video game)
http://thebestontology.com/Lost_in_Blue | Lost in Blue
http://thebestontology.com/Mass_Effect_(video_game) | Mass Effect (video game)

(rdflib.term.URIRef('http://thebestontology.com/Mass_Effect_(video_game)'), rdflib.term.Literal('Mass Effect (video game)', lang='en'))


In [185]:
query = """SELECT ?name ?ign ?metacritic
WHERE {
?game rdfs:label ?name .
?game dbp:ign ?ign .
?game dbp:mc ?metacritic .
?game dbo:releaseDate ?date
FILTER (?ign >= 90) .
FILTER (?metacritic >= 90) .
}
LIMIT 10
"""

query_result = graph.query(query)

for row in query_result:
  print(f'{row[0]} | Score: {row[1]}')

World of Warcraft | Score: 91
Uncharted 2: Among Thieves | Score: 95
Pushmo | Score: 95
LittleBigPlanet 2 | Score: 90
Section 8 (video game) | Score: 3600
Ratchet & Clank: Up Your Arsenal | Score: 96
Metal Gear Solid HD Collection | Score: 90
Gears of War 2 | Score: 95
ESPN NFL 2K5 | Score: 93
Bloodborne | Score: 91
